In [2]:
import pandas as pd
import datetime as dt
import numpy as np

In [3]:
class SBFTradingRule():
    # strategy: hold weight w_big in mkt_rf when dy > dy_cutoff,
    # weight w_small in mkt_rf when dy < dy_cutoff

    w = 0.01
    w_big = w
    w_small = w
    size_cutoff = 0
    
    # Assume 100% of portfolio liquidated each month and repurchased with new quantities
    
    # strategy-specifc columns for trades_df (variables we want to keep track of for subsequent analysis)
    # usually the variables that led the strategy to open trade in the first place
    # In this case, just size
    strategy_specific_trades_df_columns = {'size': pd.Series([], dtype='float')} 
    
    def __init__(self,portfolio_db):
        self.portfolio_db = portfolio_db
    
    # Regardless of the strategy you are implementing, this method must return
    # open_trades_df, close_trades_df
    #
    # open_trades_df is a DataFrame with all the required trades_df columns plus any custom ones for this strategy
    #     each row is a new trade the strategy wants to open
    #     method only populates the security_id (index) and quantity required columns, plus any custom columns. Rest remain NaN to be populated elsewhere
    # close_trades_df is a DataFrame that is a subset of the rows of portfolio_db.trades_df
    #     each row is an exist trade the strategy wants to close
    #     we don't need to populate any columns in this function
    def compute_trades(self,signal_df):                
        # Since we are doing 100% turnover each period, all currently open trades should be closed
        close_trades_df = self.portfolio_db.trades_df.loc[ self.portfolio_db.trades_df.loc[:,'close_datetime'].isna() ,: ].copy()

        # create empty open_trades_df
        open_trades_df = self.empty_trades_df()

        # get current dy
        current_size = signal_df['size'].iloc[-1]
        if( current_size > self.size_cutoff ):
            w_current = self.w_big
        else:
            w_current = self.w_small

        dollars_to_trade = self.portfolio_db.current_nav()*w_current

        ##### NEED TO FIGURE OUT WHERE TO ADD PRICE TO SIGNAL_DF
        quantity_to_trade = dollars_to_trade / signal_df['prc'].iloc[-1]

        # Add new row to the open_trades_df with quantity and security_id
        open_trades_df.loc[0] = {'security_id':signal_df['security_id'].iloc[-1], 'quantity':quantity_to_trade}
        
        return open_trades_df, close_trades_df
    
    # Returns an empty trades_df
    # Used so we know the right columns to populate when creating a trades_df else
    def empty_trades_df(self):
        return pd.concat([self.portfolio_db.empty_trades_df(), pd.DataFrame(self.strategy_specific_trades_df_columns)], axis=1)
        